<div style="position: relative; text-align: center;">
  <img src="imagenes/portada.png" alt="INE" width="50%">
</div> <br><br>


<p style="text-align: center; font-size: 20px;"><u>ÍNDICE</u></p>

<span style="font-size: 20px;">

1. **Introducción**

2. **Importación de paquetes**<br>

3. **Funciones**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3.1. Función de limpieza<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3.2. Funcion de inicialización<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3.3. Función de extracción de información<br>

5.  **Proceso de extracción de información**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4.1. Inicializo variables<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4.2. Extracción de información<br>

6. **Comprobación de resultados**<br>

</span>

# 1. Introducción

# 2. Importación de paquetes

In [1]:
import os
import glob
import json
import pickle
from tqdm import tqdm

import pandas as pd

import re
from pypdf import PdfReader
from langchain.chat_models import ChatOpenAI
from langchain_groq import ChatGroq
from typing import List, Optional
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

# 3. Funciones

## 3.1. Función de limpieza

In [3]:
def bill_cleaner(path):

    """
    Función que devuelve el texto procesado de una factura.

    Input:
        - path(str): Ruta de la factura.pdf
    
    Output:
        - texto_limpio (str)
    
    """

    factura = PdfReader(path)
    
    texto_factura = ""
    for pagina in factura.pages:
        texto_factura += pagina.extract_text()
    
    # Elimino saltos de linea
    texto_limpio = re.sub(r"\s+", " ", texto_factura).strip()
    
    # Elimino puntos
    texto_limpio = re.sub(r"\.+", "", texto_limpio)
    
    # Elimino espacios multiples
    texto_limpio = re.sub(r"\s+", " ", texto_limpio)

    return texto_limpio

## 3.2. Funcion de inicialización

In [4]:
def inicializar():

    data_path = input("Introduce la ruta donde se encuentran las facturas de las que deseas extraer información: ")

    model = input("Selecciona OPENAI(1) o LLaMa(2): ")

    if model == "1":
        nombre = input("Introduce el nombre asociado a la API key de OPENAI que aparece en tu archivo .ENV: ")
        api_key = os.getenv(nombre)
    if model == "2":
        nombre = input("Introduce el nombre asociado a la API key de GROQ que aparece en tu archivo .ENV: ")
        api_key = os.getenv(nombre)

    return data_path, model, api_key

## 3.3. Función de extracción de información

In [25]:
def information_extractor(path, model, api_key):

    """
    Función que extrae la información requerida de una factura de luz guardandola en formato JSON.

    Input:
        - path(str): Ruta donde se alojan las factura en formato PDF
        - api_key(str): La api key de OPENAI o GROQ
    
    Output:
        - archivo JSON con la información requerida de la factura.
    
    """
    
    # Defino el LLM:
    if model == "1":
        llm = ChatOpenAI(
            model_name="gpt-3.5-turbo",
            temperature=0,
            openai_api_key= api_key)
        
    if model == "2":
        llm = ChatGroq(model="llama3-70b-8192",
                       groq_api_key= api_key)   


    # Cargo el esquema:
    with open("utils/schema.pkl", "rb") as f:
        schema = pickle.load(f)

    # Cargo cadena
    chain = create_extraction_chain(llm, schema, encoder_or_encoder_class="json")
    
    
    
    # Busco las facturas en PDF en la ruta proporcionada:
    
    facturas = glob.glob(os.path.join(path, "*.pdf"))[710:715]

    # Obtengo la información requerida de cada factura:
    
    if not os.path.exists("extracted_information"):
        os.makedirs("extracted_information")
    
    barra_progreso = tqdm(total= len(facturas), desc="Progreso", unit="archivo")
    
    for path_factura in facturas:
        file_name = path_factura.split("\\")[1].split(".")[0] + ".json"
        information_path = "extracted_information\\"

        ###################################### Limpieza texto ######################################
        
        texto_factura = bill_cleaner(path_factura)

        ###################################### Extracción de información requerida ######################################
        
        informacion_factura = chain.invoke(input= texto_factura)["text"]["data"]["informacion_factura"][0]

        ###################################### Guardo en formato JSON la informacion ######################################
        with open(information_path + file_name, "w") as json_file:
            json.dump(informacion_factura, json_file, indent=4)

        
        barra_progreso.update(1)

    barra_progreso.close()

# 4. Proceso de extracción de información

## 4.1. Inicializo variables

<span style="font-size:larger;">

* **data_path**: Es la carperta donde se encuentran las facturas en pdf
* **openai_api_key**: La clave de la API de OPENAI en el archivo .env

Ejemplo:

- <u>*Usando gpt-3.5-turbo*</u>:

```api_key= os.getenv("OPENAI_API_KEY")
data_path = "data/"
```

- <u>*Usando LLaMa 3-70b*</u>:

```
api_key= os.getenv("GROQ_API_KEY")
data_path = "data/"
```

<br>

<u>**Ejecuta la siguiente linea de código para inicializar las variables y comenzar con el proceso de extracción de información**</u>:

   
</span>

In [18]:
data_path, model, api_key = inicializar()

Introduce la ruta donde se encuentran las facturas de las que deseas extraer información:  data/
Selecciona OPENAI(1) o LLaMa(2):  1
Introduce el nombre asociado a la API key de OPENAI que aparece en tu archivo .ENV:  OPENAI_API_KEY


## 4.2. Extracción de información

<span style="font-size:larger;">

La <u>**siguiente linea de código**</u> ejecutará la función para <u>**extraer información**</u> de las facturas, creando la carpeta *extracted_information* donde se guardará la información de cada una de las faturas en formato JSON, llevando el mismo nombre de la factura correspondiente.

</span>

In [21]:
information_extractor(data_path, model, api_key)

Progreso: 100%|██████████| 5/5 [00:23<00:00,  4.77s/archivo]


# 5. Comprobación de resultados

In [10]:
extracted_info_path = "extracted_information\\"
data = pd.read_csv(data_path + "df_finetuning.csv")

In [32]:
factura_0 = data.loc[741].output.replace("'", '"')
json.loads(factura_0)

{'nombre_cliente': 'ANÍBAL VILLAR BAÑOS',
 'dni_cliente': '98523853H',
 'calle_cliente': 'Camino de Cerro Mesa',
 'cp_cliente': '43364',
 'población_cliente': 'Mont-ral',
 'provincia_cliente': 'Tarragona',
 'nombre_comercializadora': 'HEFAMECOM, S.A.',
 'cif_comercializadora': 'A73027930',
 'dirección_comercializadora': 'CARRETERA DE ABANILLA, KM. 158',
 'cp_comercializadora': '30140',
 'población_comercializadora': 'SANTOMERA',
 'provincia_comercializadora': 'Murcia',
 'número_factura': 'FJ8239550189',
 'inicio_periodo': '02.10.2002',
 'fin_periodo': '01.12.2002',
 'importe_factura': '160,74',
 'fecha_cargo': '06.12.2002',
 'consumo_periodo': 520,
 'potencia_contratada': '3,206'}

In [33]:
with open(extracted_info_path + f"factura_{741}.json", "r", encoding= "utf-8") as archivo:
        factura_0_test = json.load(archivo)

In [34]:
factura_0_test

{'nombre_cliente': 'ANÍBAL VILLAR BAÑOS',
 'dni_cliente': '98523853H',
 'calle_cliente': 'Camino de Cerro Mesa',
 'cp_cliente': '43364',
 'población_cliente': 'Mont -ral',
 'provincia_cliente': 'Tarragona',
 'nombre_comercializadora': 'HEFAMECOM, SA',
 'cif_comercializadora': 'A73027930',
 'dirección_comercializadora': 'CARRETERA DE ABANILLA, KM 158',
 'cp_comercializadora': '30140',
 'población_comercializadora': 'Murcia',
 'provincia_comercializadora': 'Murcia',
 'número_factura': 'FJ8239550189',
 'inicio_periodo': '02.10.2002',
 'fin_periodo': '01.12.2002',
 'importe_factura': '160,74',
 'fecha_cargo': '06.12.2002',
 'consumo_periodo': '520',
 'potencia_contratada': '3,206'}